In [4]:
from all_formulas import *
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import ParameterGrid
import pandas as pd

In [5]:
b = 7.61
init = [1,0]
a0 = 3
q0 = 1-1e-6
n = 1
tim = 5

In [6]:
times_clearance = [120,60,30,20,8]
m_values = [10000,12000,20000]
times_treatment = [0,5]
q1_values = [1-3e-6,1-5e-6]

In [7]:
param_dict = {'times_clearance':times_clearance, 'm':m_values, 'times_treatment':times_treatment, 'q1':q1_values}
param_grid = list(ParameterGrid(param_dict))

In [8]:
def totalV_wholeinfection_full(b, a0, a1, T, q0, q1, m):
    V_plus = (1/(b*(q0**m)-a0))*(-1+np.exp(T*(b*(q0**m)-a0)))
    V_minus =  (1/(a1-b*(q1**m)))*np.exp(T*(b*(q0**m)-a0))
    return(V_plus+V_minus)

def totalX_wholeinfection_full(b, a0, a1, T, q0, q1, m, n):
    X_plus = (1/(b*(q0**(m+n))-a0))*(-1+np.exp(T*(b*(q0**(m+n))-a0)))
    X_minus =  (1/(a1-b*(q1**(m+n))))*np.exp(T*(b*(q0**(m+n))-a0))
    return(X_plus+X_minus)

def totalY_wholeinfection_full(b, a0, a1, T, q0, q1, m, n):
    X = totalX_wholeinfection_full(b, a0, a1, T, q0, q1, m, n)
    V = totalV_wholeinfection_full(b, a0, a1, T, q0, q1, m)
    Y = V-X
    return(Y)

def calc_ERF_full_cumul(b, a0, a1, T, q0, q1, m, n):
    Y_notreat = totalY_wholeinfection_full(b, a0, a1, T, q0, q0, m, n)
    Y_treat = totalY_wholeinfection_full(b, a0, a1, T, q1, q1, m, n)
    return(Y_treat/Y_notreat)

def calc_IRF_full_cumul(b, a0, a1, T, q0, q1, m):
    V_notreat = totalV_wholeinfection_full(b, a0, a1, T, q0, q0, m)
    V_treat = totalV_wholeinfection_full(b, a0, a1, T, q1, q1, m)
    return(V_treat/V_notreat)

In [9]:
all_results = []

for p in range(len(param_grid)):
    
    print(p)
    
    time_c = param_grid[p]['times_clearance']
    mval = param_grid[p]['m']
    q1val = param_grid[p]['q1']
    time_tr = param_grid[p]['times_treatment']
    
    # calculate value of a1 up to 0.01
    a1val = np.round(b-np.log(1e-4)/time_c,2)
    
    # calculate V,Y and ERF,IRF with Euler
    
    no_treatment_sim = simulation(init,b,a0,a1val,q0,q0,mval,n,tim,time_tr)*1e-3
    with_treatment_sim = simulation(init,b,a0,a1val,q0,q1val,mval,n,tim,time_tr)*1e-3
    
    Y_cumul_notreatment = np.sum(no_treatment_sim[:,1])
    Y_cumul_withtreatment = np.sum(with_treatment_sim[:,1])
    
    ERF = Y_cumul_withtreatment/Y_cumul_notreatment
    
    V_cumul_notreatment = np.sum(no_treatment_sim)
    V_cumul_withtreatment = np.sum(with_treatment_sim)
    
    IRF = V_cumul_withtreatment/V_cumul_notreatment
    
    # formulas
    
    Y_cumul_notreatment_formula = totalY_wholeinfection_full(b, a0, a1val, tim, q0, q0, mval, n)
    
    if time_tr == 0:
        Y_cumul_treatment_formula = totalY_wholeinfection_full(b, a0, a1val, tim, q1val, q1val, mval, n)
    else:
        Y_cumul_treatment_formula = totalY_wholeinfection_full(b, a0, a1val, tim, q0, q1val, mval, n)
        
    ERF_formula = Y_cumul_treatment_formula/Y_cumul_notreatment_formula
    
    V_cumul_notreatment_formula = totalV_wholeinfection_full(b, a0, a1val, tim, q0, q0, mval)
    
    if time_tr == 0:
        V_cumul_treatment_formula = totalV_wholeinfection_full(b, a0, a1val, tim, q1val, q1val, mval)
    else:
        V_cumul_treatment_formula = totalV_wholeinfection_full(b, a0, a1val, tim, q0, q1val, mval)
    
    IRF_formula = V_cumul_treatment_formula/V_cumul_notreatment_formula
    
    
    results = {'time_clearance':time_c, 'm':mval, 'q1':q1val, 'time_tr':time_tr, 'a1':a1val, 
               'Y_notreat':Y_cumul_notreatment, 'Y_notreat_formula':Y_cumul_notreatment_formula, 
               'V_notreat':V_cumul_notreatment, 'V_notreat_formula':V_cumul_notreatment_formula, 
               'Y_treat':Y_cumul_withtreatment, 'Y_treat_formula':Y_cumul_treatment_formula,
               'V_treat':V_cumul_withtreatment, 'V_treat_formula':V_cumul_treatment_formula,
               'ERF':ERF, 'ERF_formula':ERF_formula,
               'IRF':IRF, 'IRF_formula':IRF_formula}
    
    all_results.append(results)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [10]:
df_test = pd.DataFrame(all_results)

In [11]:
df_test.to_excel('results_table.xlsx')